In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim import corpora, models
from nltk.tokenize import word_tokenize
import altair as alt
import os
import re
from nltk.util import ngrams
from itertools import chain

import numpy as np
import warnings
warnings.simplefilter("ignore")

df = pd.read_csv('Data_mesh_publications_cleaned.csv')
df_sample = df.copy()



C:\Users\sitas\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\sitas\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\sitas\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
df_sample

,Filename,Content,Publisher,Year,Publication_type
0,Breaking Down Data Silos Data Mesh to Achieve ...,abstract data localization law becoming make h...,IEEE,2023,conference
1,Decentralized Data Governance as Part of a Dat...,abstract data socio technical decentralized an...,IEEE,2023,conference
2,Enterprise Data Strategy A Decentralized Data ...,abstract enterprise experience exponential gro...,IEEE,2022,conference
3,Finding Your Way Through the Jungle of Big Dat...,abstract paper present systematic ofcommon ana...,IEEE,2021,conference
4,CoK A Survey of Privacy Challenges in Relation...,growing volume data appear multiple dis tribut...,Springer,2022,conference
5,Converging Data Mesh and Microservice Principl...,company invested driven design sup port data d...,Springer,2023,conference
6,Data Integration Revitalized From Data Warehou...,year data integration architecture evolved fro...,Springer,2023,conference
7,Data Mesh as Distributed Data Platform for Lar...,rapid increase data volume last decade promote...,Springer,2023,conference
8,Data Fabric and Data Mesh Approaches with AI,look back data architecture developed response...,Springer,2023,book
9,Cow Mesh a data-mesh architecture to unify dai...,dairy economically signi cant industry caters ...,ResearchGate,2023,journal


In [3]:
def find_topic_words_lsi(df, num_topics=36, num_words=10):
    """
    Find topic words for each content in the DataFrame using LSI.
    
    Parameters:
    - df: DataFrame with a 'Content' column containing preprocessed (tokenized) texts.
    - num_topics: Number of topics to extract.
    - num_words: Number of words to show for each topic.
    
    Returns:
    - topics_df: DataFrame with topics and their words for each document.
    """
    tokenize_content = df['Content'].apply(lambda x: word_tokenize(x.lower()))

    dictionary = corpora.Dictionary(tokenize_content)

    corpus = [dictionary.doc2bow(text) for text in tokenize_content]
    
    # Create the LSI model
    tfidf = models.TfidfModel(corpus)  
    corpus_tfidf = tfidf[corpus]
    lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=num_topics)

    topics_df = pd.DataFrame()
    for n, doc in enumerate(corpus_tfidf):
        doc_topics = lsi_model[doc]
        topic_num, prop_topic = sorted(doc_topics, key=lambda x: abs(x[1]), reverse=True)[0]  
        wp = lsi_model.show_topic(topic_num, topn=num_words)
        topic_keywords = ", ".join([word for word, prob in wp])
        word_probs = [(word, round(prob, 4)) for word, prob in wp]  
        topics_df = topics_df.append(pd.Series([int(topic_num), topic_keywords, word_probs]), ignore_index=True)
    
    topics_df.columns = ['Dominant_Topic', 'Topic_Keywords','Word_Probabilities']
    return topics_df



In [4]:

topics_df = find_topic_words_lsi(df)
print(topics_df)



    Dominant_Topic                                     Topic_Keywords  \
0                6  localization, region, india, port, zone, india...   
1               14  port, ds, composition, aligned, ai, dl, cdm, m...   
2                5  localization, zone, port, utc, restriction, pi...   
3                3  dwh, archimate, gartner, vault, paradigm, fabr...   
4                9  dairy, quasi, farm, erential, anonymisation, c...   
5               11  componentization, contribution, netflix, analy...   
6               12  ds, smart, monitoring, modal, dairy, medical, ...   
7                8  gray, dairy, farm, id, cowmesh, cow, semantic,...   
8                0  interviewee, interview, bp, smart, product, or...   
9                4  dairy, farm, quasi, semantic, cowmesh, cow, er...   
10              10  gray, id, composite, layered, rq, dairy, farm,...   
11               1  interviewee, smart, bp, city, motivational, in...   
12              15  ai, ds, dl, cdm, contemporary, 

In [5]:
merged_df = pd.concat([df_sample, topics_df], axis=1)
result_data = []




In [6]:
merged_df

,Filename,Content,Publisher,Year,Publication_type,Dominant_Topic,Topic_Keywords,Word_Probabilities
0,Breaking Down Data Silos Data Mesh to Achieve ...,abstract data localization law becoming make h...,IEEE,2023,conference,6,"localization, region, india, port, zone, india...","[(localization, -0.7619), (region, -0.1381), (..."
1,Decentralized Data Governance as Part of a Dat...,abstract data socio technical decentralized an...,IEEE,2023,conference,14,"port, ds, composition, aligned, ai, dl, cdm, m...","[(port, -0.3504), (ds, -0.1629), (composition,..."
2,Enterprise Data Strategy A Decentralized Data ...,abstract enterprise experience exponential gro...,IEEE,2022,conference,5,"localization, zone, port, utc, restriction, pi...","[(localization, 0.4572), (zone, 0.1516), (port..."
3,Finding Your Way Through the Jungle of Big Dat...,abstract paper present systematic ofcommon ana...,IEEE,2021,conference,3,"dwh, archimate, gartner, vault, paradigm, fabr...","[(dwh, -0.3385), (archimate, -0.2648), (gartne..."
4,CoK A Survey of Privacy Challenges in Relation...,growing volume data appear multiple dis tribut...,Springer,2022,conference,9,"dairy, quasi, farm, erential, anonymisation, c...","[(dairy, -0.2718), (quasi, 0.2605), (farm, -0...."
5,Converging Data Mesh and Microservice Principl...,company invested driven design sup port data d...,Springer,2023,conference,11,"componentization, contribution, netflix, analy...","[(componentization, -0.1262), (contribution, 0..."
6,Data Integration Revitalized From Data Warehou...,year data integration architecture evolved fro...,Springer,2023,conference,12,"ds, smart, monitoring, modal, dairy, medical, ...","[(ds, 0.2555), (smart, -0.1537), (monitoring, ..."
7,Data Mesh as Distributed Data Platform for Lar...,rapid increase data volume last decade promote...,Springer,2023,conference,8,"gray, dairy, farm, id, cowmesh, cow, semantic,...","[(gray, -0.2016), (dairy, -0.1978), (farm, -0...."
8,Data Fabric and Data Mesh Approaches with AI,look back data architecture developed response...,Springer,2023,book,0,"interviewee, interview, bp, smart, product, or...","[(interviewee, 0.1959), (interview, 0.1386), (..."
9,Cow Mesh a data-mesh architecture to unify dai...,dairy economically signi cant industry caters ...,ResearchGate,2023,journal,4,"dairy, farm, quasi, semantic, cowmesh, cow, er...","[(dairy, -0.2912), (farm, -0.2629), (quasi, -0..."


In [7]:
for _, row in merged_df.iterrows():
    Filename = row['Filename']
    topic_words = row['Word_Probabilities']


    for word, prob in topic_words:
        result_data.append({'Filename': Filename, 'topic_word': word, 'prob': prob})


result_df = pd.DataFrame(result_data, columns=['Filename', 'topic_word', 'prob'])

print(result_df)

                                              Filename    topic_word    prob
0    Breaking Down Data Silos Data Mesh to Achieve ...  localization -0.7619
1    Breaking Down Data Silos Data Mesh to Achieve ...        region -0.1381
2    Breaking Down Data Silos Data Mesh to Achieve ...         india -0.1317
3    Breaking Down Data Silos Data Mesh to Achieve ...          port  0.1008
4    Breaking Down Data Silos Data Mesh to Achieve ...          zone  0.0932
..                                                 ...           ...     ...
205  Utilization of Data Mesh Framework as a Part o...  organization  0.1105
206  Utilization of Data Mesh Framework as a Part o...  motivational  0.1030
207  Utilization of Data Mesh Framework as a Part o...        fabric  0.0966
208  Utilization of Data Mesh Framework as a Part o...     lakehouse  0.0911
209  Utilization of Data Mesh Framework as a Part o...          city  0.0891

[210 rows x 3 columns]


In [21]:
# Select the top terms for each PDF
top_terms= result_df.groupby('Filename').head(10)

# Add a small random value to 'idf_value' for better visualization
top_terms['prob'] = top_terms['prob'] + np.random.rand(top_terms.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Filename:N',
    color=alt.Color('prob:Q', scale=alt.Scale(scheme='viridis')),
    text='topic_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("prob", order="descending")],
    groupby=["Filename"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='prob:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle',fontWeight='bold').encode(
    text='topic_word:N',
    color=alt.condition(alt.datum.prob >= -0.2, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_lsi= (heatmap + text).properties(width=1000)





In [22]:
chart_lsi

alt.LayerChart(...)

In [10]:
result_data_publication_type = []

# Iterate through each row in the original DataFrame
for _, row in merged_df.iterrows():
    Publication_type = row['Publication_type']
    topic_words = row['Word_Probabilities']

    # Iterate through each tuple in the list of topic_words
    for word, idf_value in topic_words:
        result_data_publication_type.append({'Publication_type': Publication_type, 'topic_word': word, 'prob': idf_value})

# Create the result DataFrame from the list
result_df_publication_type = pd.DataFrame(result_data_publication_type, columns=['Publication_type', 'topic_word', 'prob'])

# Print the result DataFrame
result_df_publication_type

,Publication_type,topic_word,prob
0,conference,localization,-0.7619
1,conference,region,-0.1381
2,conference,india,-0.1317
3,conference,port,0.1008
4,conference,zone,0.0932
...,...,...,...
205,master thesis,organization,0.1105
206,master thesis,motivational,0.1030
207,master thesis,fabric,0.0966
208,master thesis,lakehouse,0.0911


In [18]:
# Select the top terms for each PDF
top_terms_df_publication_type = result_df_publication_type.groupby('Publication_type').head(10)

# Add a small random value to 'idf_value' for better visualization
top_terms_df_publication_type['prob'] = top_terms_df_publication_type['prob'] + np.random.rand(top_terms_df_publication_type.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms_df_publication_type).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Publication_type:N',
    color=alt.Color('prob:Q', scale=alt.Scale(scheme='viridis')),
    text='topic_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("prob", order="descending")],
    groupby=["Publication_type"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='prob:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle',fontWeight='bold').encode(
    text='topic_word:N',
    color=alt.condition(alt.datum.prob >= -0.2, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_lda_publication_type = (heatmap + text).properties(width=2000)
chart_lda_publication_type

alt.LayerChart(...)

In [19]:
result_data_publisher = []

# Iterate through each row in the original DataFrame
for _, row in merged_df.iterrows():
    Publisher = row['Publisher']
    topic_words = row['Word_Probabilities']

    # Iterate through each tuple in the list of topic_words
    for word, idf_value in topic_words:
        result_data_publisher.append({'Publisher': Publisher, 'topic_word': word, 'prob': idf_value})

# Create the result DataFrame from the list
result_df_publisher = pd.DataFrame(result_data_publisher)


result_df_publisher

,Publisher,topic_word,prob
0,IEEE,localization,-0.7619
1,IEEE,region,-0.1381
2,IEEE,india,-0.1317
3,IEEE,port,0.1008
4,IEEE,zone,0.0932
...,...,...,...
205,miscellaneous,organization,0.1105
206,miscellaneous,motivational,0.1030
207,miscellaneous,fabric,0.0966
208,miscellaneous,lakehouse,0.0911


In [20]:
# Select the top terms for each PDF
top_terms_df_publisher = result_df_publisher.groupby('Publisher').head(10)

# Add a small random value to 'idf_value' for better visualization
top_terms_df_publisher['c'] = top_terms_df_publisher['prob'] + np.random.rand(top_terms_df_publisher.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms_df_publisher).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Publisher:N',
    color=alt.Color('prob:Q', scale=alt.Scale(scheme='viridis')),
    text='topic_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("prob", order="descending")],
    groupby=["Publisher"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='prob:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle', fontWeight='bold').encode(
    text='topic_word:N',
    color=alt.condition(alt.datum.prob >= -0.2, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_lda_publisher = (heatmap + text).properties(width=1500)
chart_lda_publisher

alt.LayerChart(...)

In [14]:
result_data_date = []

In [15]:
for _, row in merged_df.iterrows():
    Year = row['Year']
    topic_words = row['Word_Probabilities']


    for word, prob in topic_words:
        result_data_date.append({'Year': Year, 'topic_word': word, 'prob': prob})


result_data_date = pd.DataFrame(result_data_date, columns=['Year', 'topic_word', 'prob'])

result_data_date

,Year,topic_word,prob
0,2023,localization,-0.7619
1,2023,region,-0.1381
2,2023,india,-0.1317
3,2023,port,0.1008
4,2023,zone,0.0932
...,...,...,...
205,2021,organization,0.1105
206,2021,motivational,0.1030
207,2021,fabric,0.0966
208,2021,lakehouse,0.0911


In [17]:
# Select the top terms for each PDF
top_terms_df_dates = result_data_date.groupby('Year').head(10)

# Add a small random value to 'idf_value' for better visualization
top_terms_df_dates['prob'] = top_terms_df_dates['prob'] + np.random.rand(top_terms_df_dates.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms_df_dates).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Year:N',
    color=alt.Color('prob:Q', scale=alt.Scale(scheme='viridis')),
    text='topic_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("prob", order="descending")],
    groupby=["Year"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='prob:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle',fontWeight='bold').encode(
    text='topic_word:N',
    color=alt.condition(alt.datum.prob >= -0.2, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_lsi_date = (heatmap + text).properties(width=1500)
chart_lsi_date

alt.LayerChart(...)